In [1]:
%load_ext autoreload
%autoreload 2
import sys 
if '/Users/ericliu/Desktop/Latent-Dirichilet-Allocation' not in sys.path: 
    sys.path.append('/Users/ericliu/Desktop/Latent-Dirichilet-Allocation')
import torch as tr 
import numpy as np 
import pandas as pd 
from collections import defaultdict
from pprint import pprint

/Users/ericliu/Desktop/Latent-Dirichilet-Allocation/.venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.lda_model import LDASmoothed 
from src.generator import doc_generator 
from src.utils import (
    get_vocab_from_docs, 
    get_np_wct, 
    data_loader,
    text_pipeline, 
    process_documents,
) 
from src.text_pre_processor import (
    remove_accented_chars, 
    remove_special_characters, 
    remove_punctuation,
    remove_extra_whitespace_tabs,
    remove_stopwords,
)
from pprint import pprint 
import copy 

In [3]:
gen = doc_generator(
    M = 3,
    L = 20, 
    topic_prior = tr.tensor([1,1,1,1,1], dtype=tr.double)
)

In [4]:
gen.beta

tensor([[0.1131, 0.0589, 0.0824, 0.1060, 0.0471, 0.0824, 0.0294, 0.1060, 0.0059,
         0.0047, 0.0353, 0.0059, 0.0118, 0.0035, 0.0012, 0.0012, 0.0118, 0.0118,
         0.0236, 0.0236, 0.0177, 0.0177, 0.0041, 0.0047, 0.0353, 0.0059, 0.0118,
         0.0353, 0.0177, 0.0118, 0.0029, 0.0035, 0.0029, 0.0012, 0.0353, 0.0029,
         0.0059, 0.0118, 0.0029, 0.0029],
        [0.0012, 0.0047, 0.0047, 0.0030, 0.0355, 0.0118, 0.0237, 0.0030, 0.0948,
         0.0592, 0.0592, 0.1066, 0.0474, 0.1066, 0.0948, 0.1126, 0.0237, 0.0059,
         0.0059, 0.0118, 0.0178, 0.0178, 0.0041, 0.0047, 0.0047, 0.0237, 0.0118,
         0.0296, 0.0059, 0.0237, 0.0030, 0.0036, 0.0030, 0.0012, 0.0036, 0.0030,
         0.0047, 0.0118, 0.0030, 0.0030],
        [0.0014, 0.0057, 0.0043, 0.0036, 0.0143, 0.0071, 0.0286, 0.0036, 0.0071,
         0.0043, 0.0214, 0.0014, 0.0428, 0.0043, 0.0143, 0.0014, 0.0500, 0.0999,
         0.0999, 0.0571, 0.0857, 0.0857, 0.1285, 0.1285, 0.0057, 0.0014, 0.0143,
         0.0071, 0.0071, 

In [5]:
gen.alpha

Dirichlet(concentration: torch.Size([5]))

In [6]:
gen.theta.shape

torch.Size([3, 5])

In [7]:
gen.theta

tensor([[0.2737, 0.1182, 0.0215, 0.3038, 0.2829],
        [0.0219, 0.4398, 0.1990, 0.3358, 0.0035],
        [0.0611, 0.0508, 0.4054, 0.3969, 0.0858]], dtype=torch.float64)

In [8]:
docs = gen.generate_doc()

Document: 0 | word: 0 -> topic: health -> word: decongestant
Document: 0 | word: 1 -> topic: sport -> word: immunology
Document: 0 | word: 2 -> topic: health -> word: genetics
Document: 0 | word: 3 -> topic: science -> word: physical
Document: 0 | word: 4 -> topic: science -> word: research
Document: 0 | word: 5 -> topic: sport -> word: copyright
Document: 0 | word: 6 -> topic: science -> word: energy
Document: 0 | word: 7 -> topic: sport -> word: football
Document: 0 | word: 8 -> topic: science -> word: allergy
Document: 0 | word: 9 -> topic: law -> word: evidence
Document 0: decongestant immunology genetics physical research copyright energy football allergy evidence

Document: 1 | word: 0 -> topic: art -> word: concert
Document: 1 | word: 1 -> topic: health -> word: decongestant
Document: 1 | word: 2 -> topic: health -> word: bruise
Document: 1 | word: 3 -> topic: science -> word: quantum
Document: 1 | word: 4 -> topic: sport -> word: evidence
Document: 1 | word: 5 -> topic: sport -

In [9]:
docs

{0: 'decongestant immunology genetics physical research copyright energy football allergy evidence',
 1: 'concert decongestant bruise quantum evidence physical infection physical Olympic Craftsmanship',
 2: 'accuse asymmetrical asymmetrical evidence research asymmetrical infection Symmetrical genetics research'}

In [10]:
#docs_raw_dict, raw_word_2_idx, raw_idx_2_word = data_loader('ap')

In [11]:
result = process_documents(docs, sample=True)

There are 3 documents in the dataset after processing
On average estimated document length is 10.0 words per document after processing
There are 19 unique vocab in the corpus after processing


In [12]:
result['vocab_doc_count_dict']

{'decongestant': [1, 1, 0],
 'immunology': [1, 0, 0],
 'genetics': [1, 0, 1],
 'physical': [1, 2, 0],
 'research': [1, 0, 2],
 'copyright': [1, 0, 0],
 'energy': [1, 0, 0],
 'football': [1, 0, 0],
 'allergy': [1, 0, 0],
 'evidence': [1, 1, 1],
 'concert': [0, 1, 0],
 'bruise': [0, 1, 0],
 'quantum': [0, 1, 0],
 'infection': [0, 1, 1],
 'Olympic': [0, 1, 0],
 'Craftsmanship': [0, 1, 0],
 'accuse': [0, 0, 1],
 'asymmetrical': [0, 0, 3],
 'Symmetrical': [0, 0, 1]}

In [13]:
result['vocab_doc_count_array']

array([[1., 1., 0.],
       [1., 0., 0.],
       [1., 0., 1.],
       [1., 2., 0.],
       [1., 0., 2.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 1., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 3.],
       [0., 0., 1.]])

In [65]:
lda = LDASmoothed(
    docs = result['documents'],
    num_topics = 5, 
    word_ct_dict = result['vocab_doc_count_dict'], 
    word_ct_array = result['vocab_doc_count_array'],
)
#LDASmoothed(docs, 3, wct_dict)

Topic Dirichlet Prior, Alpha
torch.Size([5])

Exchangeable Word Dirichlet Prior, Eta 
1

Var Inf - Word Dirichlet prior, Lambda
torch.Size([5, 19])

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([3, 5])

loop phi
looped
double
Var -Inf - Word wise Topic Multinomial/Categorical, Phi
torch.Size([3, 19, 5])


In [66]:
lda._alpha_

tensor([1.0747, 1.1815, 0.9303, 0.9676, 0.8724], dtype=torch.float64)

In [67]:
lda._gamma_

tensor([[4.8747, 4.9815, 4.7303, 4.7676, 4.6724],
        [4.8747, 4.9815, 4.7303, 4.7676, 4.6724],
        [4.8747, 4.9815, 4.7303, 4.7676, 4.6724]], dtype=torch.float64)

In [68]:
lda._phi_

tensor([[[0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.2000, 0.2000, 0.2000, 0.2000, 0.2000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.2000, 0.2000, 0.2

In [84]:
lda = LDASmoothed(
    docs = result['documents'],
    num_topics = 5, 
    word_ct_dict = result['vocab_doc_count_dict'], 
    word_ct_array = result['vocab_doc_count_array'],
) 

Topic Dirichlet Prior, Alpha
torch.Size([5])

Exchangeable Word Dirichlet Prior, Eta 
1

Var Inf - Word Dirichlet prior, Lambda
torch.Size([5, 19])

Var Inf - Topic Dirichlet prior, Gamma
torch.Size([3, 5])

loop phi
looped
double
Var -Inf - Word wise Topic Multinomial/Categorical, Phi
torch.Size([3, 19, 5])


In [87]:
np.random.seed(42)

lda._alpha_ = np.random.gamma(shape=100, scale=0.01, size=lda.K)
lda._gamma_ = lda._alpha_ + np.ones((lda.M, lda.K)) * result['vocab_to_idx']/ lda.K 

lda._alpha_ = tr.tensor(lda._alpha_, dtype=tr.double)
lda._gamma_ = tr.tensor(lda._gamma_, dtype=tr.double)

print(lda._alpha_)
print(lda._gamma_)

tensor([1.0471, 0.9829, 0.9735, 0.9735, 1.1628], dtype=torch.float64)
tensor([[4.8471, 4.7829, 4.7735, 4.7735, 4.9628],
        [4.8471, 4.7829, 4.7735, 4.7735, 4.9628],
        [4.8471, 4.7829, 4.7735, 4.7735, 4.9628]], dtype=torch.float64)


In [86]:
lda.update_alpha()

72.4192192526763|tensor([1.0471, 0.9829, 0.9735, 0.9735, 1.1628], dtype=torch.float64) -> tensor([1.7821, 1.7058, 1.6943, 1.6943, 1.9116], dtype=torch.float64)
Iteration 0, Delta Alpha = 1.6317682001101212 elbo is -278.82768267943055
72.4192192526763|tensor([1.7821, 1.7058, 1.6943, 1.6943, 1.9116], dtype=torch.float64) -> tensor([2.8356, 2.7569, 2.7449, 2.7449, 2.9664], dtype=torch.float64)
Iteration 1, Delta Alpha = 2.352649532593188 elbo is -276.0705562049756
72.4192192526763|tensor([2.8356, 2.7569, 2.7449, 2.7449, 2.9664], dtype=torch.float64) -> tensor([3.9752, 3.9027, 3.8917, 3.8917, 4.0992], dtype=torch.float64)
Iteration 2, Delta Alpha = 2.554457301341353 elbo is -274.3524646043217
72.4192192526763|tensor([3.9752, 3.9027, 3.8917, 3.8917, 4.0992], dtype=torch.float64) -> tensor([4.6827, 4.6166, 4.6068, 4.6068, 4.8002], dtype=torch.float64)
Iteration 3, Delta Alpha = 1.5887650484159113 elbo is -273.6558400029852
72.4192192526763|tensor([4.6827, 4.6166, 4.6068, 4.6068, 4.8002], dty

In [20]:
lda_trained._eta_

tensor([199.0945, 100.0088, 199.0314, 298.0643, 297.9187, 100.0088, 100.0088,
        100.0088, 100.0088, 298.1493,  99.9743,  99.9743,  99.9743, 199.0503,
         99.9743,  99.9743,  99.9450, 297.7011,  99.9450], dtype=torch.float64)

In [21]:
lda_trained._lambda_

tensor([[199.2372, 100.0801, 199.1742, 298.2784, 298.1329, 100.0801, 100.0801,
         100.0801, 100.0801, 298.3635, 100.0456, 100.0456, 100.0456, 199.1931,
         100.0456, 100.0456, 100.0163, 297.9153, 100.0163],
        [198.7724,  99.8477, 198.7093, 297.5811, 297.4356,  99.8477,  99.8477,
          99.8477,  99.8477, 297.6662,  99.8132,  99.8132,  99.8132, 198.7282,
          99.8132,  99.8132,  99.7839, 297.2180,  99.7839],
        [199.7479, 100.3355, 199.6849, 299.0445, 298.8990, 100.3355, 100.3355,
         100.3355, 100.3355, 299.1296, 100.3010, 100.3010, 100.3010, 199.7038,
         100.3010, 100.3010, 100.2717, 298.6814, 100.2717],
        [198.9259,  99.9245, 198.8629, 297.8115, 297.6659,  99.9245,  99.9245,
          99.9245,  99.9245, 297.8966,  99.8900,  99.8900,  99.8900, 198.8818,
          99.8900,  99.8900,  99.8607, 297.4483,  99.8607],
        [198.7922,  99.8576, 198.7291, 297.6109, 297.4653,  99.8576,  99.8576,
          99.8576,  99.8576, 297.6959,  99.8231, 

In [22]:
lda_trained._phi_

tensor([[[0.2710, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2711, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2710, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2710, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2710, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2711, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2711, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2711, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2711, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.2710, 0.0386, 0.5264, 0.1154, 0.0485],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.2710, 0.0386, 0.5

In [153]:
lda_trained._gamma_

tensor([[397.5601, 918.4914, 459.1947, 821.5236, 730.4222],
        [398.1572, 919.8720, 459.8845, 822.7584, 731.5199],
        [397.5601, 918.4914, 459.1947, 821.5236, 730.4222]],
       dtype=torch.float64)